## Model Exploration for Vegetable Classification
### Introduction
The goal of this project is to build an easy-to-use web application that allows users to identify vegetables from images using a trained machine learning model. It is designed primarily for educational purposes and as a demonstration of practical image classification using AI.

Framing the Problem
This notebook's goal is to train and fine tune a Image classification model to predict images of different vegetables, then evaluate our findings based on our models results.

- This project uses a dataset from kaggle with hundreds of images of vegetables
- After this intoduction, we will explore the data, such as the number of files split between the training, test and validation folders, image dimensions, etc
- After Data Exploration, we will train, and fine tune multiple models to find which best fits our image data properly
- We will evaluate the preformance of the all models. We'll look into accuracy, confusion metric, precision, recall, F1-score, precision-recall curve for the model.
- We will explore examples in which our models failed to predict correctly.

### Data Exploration
Dataset (Source: [Vegetable Image Dataset](https://www.kaggle.com/datasets/misrakahmed/vegetable-image-dataset/data)
)
The dataset contains 21000 images unique images of 15 different classes. I have modified the dataset and reduced the classes to 10 for training purposes.

Image classes in dataset:

- Bean
- Broccoli
- Cabbage
- Capsicum
- Carrot
- Cauliflower
- Cucumber
- Potato
- Pumpkin
- Tomato
We have reduced the dataset to 2000 images for training (200 in each class), 2000 for testing, and 1000 for validation (100 in each class).

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import VGG16, EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pathlib

ModuleNotFoundError: No module named 'numpy'

In [ ]:
data_dir = pathlib.Path('data/kaggle_vegetables_small')

In [ ]:
# from tensorflow.keras.utils import image_dataset_from_directory

# train_dataset = image_dataset_from_directory(
#     "data/kaggle_vegetables_small/train",
#     image_size=(180, 180),
#     batch_size=32)
# validation_dataset = image_dataset_from_directory(
#     "data/kaggle_vegetables_small/validation",
#     image_size=(180, 180),
#     batch_size=32)
# test_dataset = image_dataset_from_directory(
#     "data/kaggle_vegetables_small/test",
#     image_size=(180, 180),
#     batch_size=32)

def load_data(data_dir, img_size=(224, 224), batch_size=32):
    train_dir = os.path.join(data_dir, "train")
    val_dir = os.path.join(data_dir, "validation")
    test_dir = os.path.join(data_dir, "test")
    
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    )
    val_test_datagen = ImageDataGenerator(rescale=1./255)
    
    train_generator = train_datagen.flow_from_directory(
        train_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical'
    )
    val_generator = val_test_datagen.flow_from_directory(
        val_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical'
    )
    test_generator = val_test_datagen.flow_from_directory(
        test_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical', shuffle=False
    )
    
    return train_generator, val_generator, test_generator

### Modeling
We explored the use of multiple models to find the most efficient one for our task

Models we investigated:

CNN: Simple and great for basic image classification tasks.
VGG16: Deep architecture and has high performance on image classification tasks.
EfficientNet: High accuracy with less parameters and lower computational costs.
ResNet50: Deep architecture and residual connections, can learn complex patterns.
MobileNet: Lightweight architecture, and can quickly classify images with high accuracy.

When these models are compiled, we're using the adam optimizer with a learning rate of 0.001 as it's one of the most efficient. We set the loss to categorical_crossentropy, which is suitable for our data multi-class classification.

In [ ]:

def build_model(model_name, input_shape, num_classes):
    if model_name == "CNN":
        model = Sequential([
            Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
            MaxPooling2D((2, 2)),
            Conv2D(64, (3, 3), activation='relu'),
            MaxPooling2D((2, 2)),
            Flatten(),
            Dense(128, activation='relu'),
            Dropout(0.5),
            Dense(num_classes, activation='softmax')
        ])
    elif model_name == "VGG16":
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
        for layer in base_model.layers:
            layer.trainable = False
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(256, activation='relu')(x)
        x = Dropout(0.5)(x)
        model = Model(inputs=base_model.input, outputs=Dense(num_classes, activation='softmax')(x))
    elif model_name == "EfficientNet":
        base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
        for layer in base_model.layers:
            layer.trainable = False
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(256, activation='relu')(x)
        x = Dropout(0.5)(x)
        model = Model(inputs=base_model.input, outputs=Dense(num_classes, activation='softmax')(x))
    elif model_name == "ResNet50":
        base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
        for layer in base_model.layers:
            layer.trainable = False
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(256, activation='relu')(x)
        x = Dropout(0.5)(x)
        model = Model(inputs=base_model.input, outputs=Dense(num_classes, activation='softmax')(x))
    elif model_name == "MobileNet":
        base_model = tf.keras.applications.MobileNet(weights='imagenet', include_top=False, input_shape=input_shape)
        for layer in base_model.layers:
            layer.trainable = False
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(256, activation='relu')(x)
        x = Dropout(0.5)(x)
        model = Model(inputs=base_model.input, outputs=Dense(num_classes, activation='softmax')(x))   
    else:
        raise ValueError("Unsupported model name")
    
    model.compile(optimizer=Adam(learning_rate=0.001), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    return model

In [ ]:
# ---------------------------------------
# Training and Evaluation
# ---------------------------------------

def train_and_evaluate(model, train_generator, val_generator, test_generator):
    callbacks = [
        ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1, min_lr=1e-6),
        EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
    ]
    history = model.fit(train_generator, epochs=20, validation_data=val_generator, callbacks=callbacks)
    
    # Evaluate on test set
    test_loss, test_acc = model.evaluate(test_generator, verbose=1)
    print(f"Test Accuracy: {test_acc:.2f}")
    
    # Generate predictions
    y_pred = np.argmax(model.predict(test_generator), axis=1)
    y_true = test_generator.classes
    
    # Detailed metrics
    report = classification_report(y_true, y_pred, target_names=list(test_generator.class_indices.keys()))
    conf_matrix = confusion_matrix(y_true, y_pred)
    return test_acc, report, conf_matrix, history

In [ ]:
# ---------------------------------------
# Main Workflow
# ---------------------------------------


train_gen, val_gen, test_gen = load_data(data_dir)

results = []
models_to_test = ["CNN", "VGG16", "EfficientNet", "ResNet50", "MobileNet"]
input_shape = (224, 224, 3)
num_classes = len(train_gen.class_indices)

for model_name in models_to_test:
    print(f"\nTraining {model_name}...\n")
    model = build_model(model_name, input_shape, num_classes)
    test_acc, report, conf_matrix, history = train_and_evaluate(model, train_gen, val_gen, test_gen)
    
    # Save results
    results.append({
        "Model": model_name,
        "Accuracy": test_acc,
        "Report": report,
        "Confusion Matrix": conf_matrix
    })
    print(report)

# Display results
for result in results:
    print(f"Model: {result['Model']}\nAccuracy: {result['Accuracy']}\n")

# Convert results to a DataFrame for better presentation
results_df = pd.DataFrame(results, columns=["Model", "Accuracy", "Report"])
print("\nModel Performance Summary:")
print(results_df[["Model", "Accuracy"]])

 precision    recall  f1-score   support

        Bean       0.70      0.83      0.76       200
    Broccoli       0.75      0.96      0.84       200
     Cabbage       0.78      0.77      0.77       200
    Capsicum       0.98      0.93      0.95       200
      Carrot       0.98      0.99      0.98       200
 Cauliflower       0.93      0.71      0.81       200
    Cucumber       0.91      0.77      0.83       200
      Potato       0.89      0.98      0.94       200
     Pumpkin       0.77      0.83      0.80       200
      Tomato       0.91      0.71      0.80       200

    accuracy                           0.85      2000
   macro avg       0.86      0.85      0.85      2000
weighted avg       0.86      0.85      0.85      2000


Training VGG16...
Model Performance Summary:
          Model  Accuracy
0           CNN    0.8480
1         VGG16    0.9850
2  EfficientNet    0.1000
3      ResNet50    0.2865
4     MobileNet    0.9985
Model: CNN

Test Accuracy: 0.8479, Precision: 0.86, Recall: 0.85, f1-Score: 0.85

Model: VGG16

Test Accuracy: 0.9850, Precision: 0.99, Recall: 0.98, f1-Score: 0.99

Model: EfficientNet

Test Accuracy: 0.1000, Precision: 0.01, Recall: 0.10, f1-Score: 0.02

Model: ResNet50

Test Accuracy: 0.2865, Precision: 0.31, Recall: 0.29, f1-Score: 0.26

Model: MobileNet

Test Accuracy: 0.9984, Precision: 1.00, Recall: 1.00, f1-Score: 1.00

Comments
The results from the reports show that the VGG16 and MobileNet models performed exceptionally well in classifying the vegetable images, with very high accuracy, precision, recall, and f1-scores. CNN also showed decent performance but not as close to VGG16 and MobileNet. EfficientNet and ResNet50 struggled, with EfficientNet performing very poorly, meaning there was an issue with our training of that model.


In [ ]:
# ---------------------------------------
# Visualization - Accuracy and Loss
# ---------------------------------------

def plot_training(history, model_name):
    plt.figure(figsize=(12, 5))

    # Plot accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title(f'{model_name} Accuracy Over Epochs')

    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.title(f'{model_name} Loss Over Epochs')

    plt.tight_layout()
    plt.show()

In [ ]:
# Plot training for each model
for model_name in models_to_test:
    print(f"Plotting training curves for {model_name}...")
    model = build_model(model_name, input_shape, num_classes)
    _, _, _, history = train_and_evaluate(model, train_gen, val_gen, test_gen)
    plot_training(history, model_name)

CNN Accuracy and Loss Plots - Test Accuracy 0.86
Accuracy Plot

Training steadily increases as the model learns from the data.

Validation increases but starts to plateau around epoch 10, indicating the model is reaching its capacity for the given task.

Loss Plot

Training decreases, showing the model is fitting the training data better with each epoch.

Validation decreases initially but then stabilizes, indicating that the model is no longer improving much on unseen data.


VGG16 Accuracy and Loss Plots - Test Accuracy 0.98
Accuracy Plot

Training increases steadily and eventually reaches close to 1.0, indicating the model over epochs is learning the patterns in the image training data effectively.

Validation follows the training data consistantly, but the accuracy is higher than the training accuracy.

Loss Plot

Training loss starts high and decreases steadily as the model learns the data, reaching a very low value near the end.

Validation loss decreases rapidly in the first few epochs and then gradually reduces, stabilizing around a low value.

EfficientNet Accuracy and Loss Plots - Test Accuracy 0.10
Accuracy Plot

Training accuracy fluctuates significantly and remains very low throughout (around 0.09–0.11), suggesting the model struggles to learn from the training data.

Validation stays constant around 0.1, indicating the model fails to generalize to unseen data.

Loss Plot

Training loss starts high at 2.35 and decreases slightly but remains close to its initial value.

Validation loss remains almost constant 2.31, showing no significant improvement.

ResNet50 Accuracy and Loss Plots - Test Accuracy 0.29
Accuracy Plot

Training increases over epochs but remains low (between 0.20–0.25).

Validation fluctuates more and is generally higher than the training accuracy.

Loss Plot

Training shows a consistent decrease, meaning the model is learning from the training data.

Validation decreases but shows some fluctuations, which suggests potential instability in validation performance.
MobileNet Accuracy and Loss Plots - Test Accuracy 1.00
Accuracy Plot

Both the training and validation accuracy quickly reach near 100% accuracy.
Loss Plot

Both the training and validation losses decrease rapidly and stabilize at low values.




In [ ]:
# ---------------------------------------
# Visualization - Confusion Matrix
# ---------------------------------------

def plot_confusion_matrix(conf_matrix, class_labels, model_name):
    plt.figure(figsize=(10, 8))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_labels, yticklabels=class_labels)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title(f'{model_name} Confusion Matrix')
    plt.show()

# Visualize confusion matrix for each model
for result in results:
    plot_confusion_matrix(result["Confusion Matrix"], list(test_gen.class_indices.keys()), result

### Conclusion
Based on our analysis of each of these models, we have found that the most sucessful one in terms of predicting images of 10 different vegetables is the MobileNet model.

MobileNet Model Overview

Nearly predicted at 100% test accuracy (0.9985), only with 3 incorrect predictions of 2000 total test images.
Predicted incorrectly for 2 images labeled as pumpkins, and 1 labeled as a capsicum.
Other accurate models:

VGG16: had a range of about 3-4 misclassified images per image class out of 200, with a test accuracy of 0.9850
CNN: Struggled in some areas but not an outright failure, this model struggled specifically with classifying images of tomato, or cauliflower (Test accuracy: 0.86)
Models that failed:

EfficientNet: Completely inaccurate for our predictions, this model would require more fine-tuning as it only classifies images as pumpkins
ResNet50: Model was all over the place in terms of predictions, test accuracy of 29